In [1]:
from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense, Dropout, Reshape, Permute
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ReLU
from keras.layers.recurrent import GRU
from keras.utils.data_utils import get_file
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
import librosa
import time
import numpy as np
np.set_printoptions(threshold=np.nan)
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


In [3]:
X_data1 = np.load('mfcc_select1_rep_bkgn.npy')
X_data2 = np.load('mfcc_select2_rep_bkgn.npy')
y_data1 = np.load('label_select1_rep_bkgn.npy')
y_data2 = np.load('label_select2_rep_bkgn.npy')

X_data = np.concatenate((X_data1,X_data2),axis=0)
y_data = np.concatenate((y_data1,y_data2),axis=0)

X_data.shape, y_data.shape

((1995, 3, 44), (1995, 2))

In [8]:
X_data = np.load('mfcc_select2_bkgn_rep.npy')
y_data = np.load('label_select2_bkgn_rep.npy')

X_data.shape, y_data.shape

((1118, 13, 44), (1118, 2))

In [23]:
from sklearn.preprocessing import StandardScaler
print(X_data[0][0])
X_data = tf.keras.utils.normalize(X_data, axis=-1, order=2)
#X_data = StandardScaler().fit_transform(X_data)
print("after")
print(X_data[0][0])

[-349.38297733 -339.46299205 -335.9631354  -332.85456749 -335.33701397
 -338.99786663 -335.77088785 -333.44439033 -329.44559082 -322.16420586
 -319.92502913 -316.64271557 -311.43076845 -308.73829721 -309.3209236
 -307.67348592 -303.83712975 -305.39611913 -310.7827827  -312.72902822
 -320.60781548 -332.39978589 -345.72288923 -354.45209134 -345.82691536
 -222.58115458 -144.63731484 -104.18134994  -94.19182443 -125.13382514
 -176.27034638 -187.67775668 -142.13757249 -106.01491192 -103.24835776
 -105.74849192 -113.21542036 -124.97840797 -130.46651078 -136.70618579
 -157.23289474 -202.31283386 -221.40212359 -224.78087842]
after
[-0.19771582 -0.1921021  -0.19012153 -0.18836239 -0.18976721 -0.19183888
 -0.19001274 -0.18869617 -0.18643325 -0.18231272 -0.18104557 -0.17918811
 -0.17623866 -0.174715   -0.1750447  -0.17411242 -0.17194142 -0.17282366
 -0.17587197 -0.17697335 -0.18143196 -0.18810503 -0.19564457 -0.20058443
 -0.19570344 -0.12595867 -0.08185025 -0.05895622 -0.05330315 -0.07081323
 -0.

In [4]:
print(X_data.shape)
X_data = np.array(X_data.reshape(X_data.shape[0], 3, 44, 1))
print(X_data.shape)
y=[]

ns=s=0
for i in range(len(y_data)):
    if y_data[i][0]==1:
        y.append('0')
        ns+=1
    else:
        y.append('1')
        s+=1

print("ns = "+str(ns))
print("s = "+str(s))
y = np.array(y)

(1995, 3, 44)
(1995, 3, 44, 1)
ns = 1542
s = 453


In [164]:
from sklearn.model_selection import train_test_split
#X_sub, X_test, y_sub, y_test = train_test_split(X_data, y_data, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.02)

In [165]:
len(X_train), len(X_test), len(y_train), len(y_test)

(1955, 40, 1955, 40)

In [166]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1955, 3, 44, 1), (40, 3, 44, 1), (1955,), (40,))

In [167]:
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential

model = Sequential()

# Model Input
input_shape=(3, 44, 1)

# Using CNN to build model
# W=(W−F+2P)/S+1
# 16 depths (3-2)/1+1 x (44-8)/1+1 = 3 x 37 x 32
model.add(Conv2D(32, (1, 8), strides=(1, 1), input_shape=input_shape))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

# 3 x 30 x 32
model.add(Conv2D(32, (1, 8), strides=(1, 1), padding="valid"))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

# 3 x 23 x 32
model.add(Conv2D(48, (1, 8), strides=(1, 1), padding="valid"))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

# 3 x 16 x 32
model.add(Conv2D(48, (1, 8), strides=(1, 1), padding="valid"))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

# 3 x 9 x 32
model.add(Conv2D(64, (1, 8), strides=(1, 1), padding="valid"))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

#print(model.summary())

model.add(Reshape((27, 64)))

# GRU block 1, 2, output
model.add(GRU(32, return_sequences=True, name='gru1'))
model.add(GRU(32, return_sequences=False, name='gru2'))
#model.add(GRU(32, return_sequences=False, name='gru3'))
model.add(Dropout(0.3))

# Output
model.add(Dense(1))
model.add(Activation('sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_120 (Conv2D)          (None, 3, 37, 32)         288       
_________________________________________________________________
activation_151 (Activation)  (None, 3, 37, 32)         0         
_________________________________________________________________
conv2d_121 (Conv2D)          (None, 3, 30, 32)         8224      
_________________________________________________________________
activation_152 (Activation)  (None, 3, 30, 32)         0         
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 3, 23, 48)         12336     
_________________________________________________________________
activation_153 (Activation)  (None, 3, 23, 48)         0         
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 3, 16, 48)         18480     
__________

In [168]:
model.compile(
    optimizer="Adam",
    loss="binary_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train,
    y=y_train,
    epochs=64,
    batch_size=64,
    validation_data= (X_test, y_test),
)

score = model.evaluate(x=X_test,y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1955 samples, validate on 40 samples
Epoch 1/64
1955/1955 [==============================] - 7s 3ms/step - loss: 0.5475 - acc: 0.7570 - val_loss: 0.4979 - val_acc: 0.7750
Epoch 2/64
1955/1955 [==============================] - 2s 825us/step - loss: 0.5133 - acc: 0.7698 - val_loss: 0.4769 - val_acc: 0.7750
Epoch 3/64
1955/1955 [==============================] - 2s 914us/step - loss: 0.4889 - acc: 0.7739 - val_loss: 0.4451 - val_acc: 0.7750
Epoch 4/64
1955/1955 [==============================] - 2s 1ms/step - loss: 0.4642 - acc: 0.7770 - val_loss: 0.3904 - val_acc: 0.8250
Epoch 5/64
1955/1955 [==============================] - 2s 1ms/step - loss: 0.4594 - acc: 0.7795 - val_loss: 0.4026 - val_acc: 0.8500
Epoch 6/64
1955/1955 [==============================] - 2s 998us/step - loss: 0.4476 - acc: 0.7898 - val_loss: 0.4063 - val_acc: 0.7750
Epoch 7/64
1955/1955 [==============================] - 2s 894us/step - loss: 0.4572 - acc: 0.7836 - val_loss: 0.4077 - val_acc: 0.8000
Epoch 8/

KeyboardInterrupt: 

In [15]:
modelName = 'rcnn-3d-select2-bkgn_rep_1_90v.h5'
model.save('models/{}'.format(modelName))

print('Model exported and finished')

Model exported and finished


In [ ]:
import librosa
#import keras
import numpy as np
import tensorflow as tf
from pydub import AudioSegment
#from pydub.silence import split_on_silence
#import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
#import matplotlib.pyplot as plt
#%matplotlib inline
#plt.style.use('ggplot')
np.set_printoptions(threshold=np.nan)

sound_file = AudioSegment.from_wav("/home/mansi/anaconda3/beproject/stutter_det/read1s2dmf/M_0048_11y1m_2/M_0048_11y1m_2.wav")

audio_chunks = sound_file[::1000]

mfcc_array = []
print("ff")
print(mfcc_array)
for i, chunk in enumerate(audio_chunks):
    chunkfile = "chunks_test/chunk{0}.wav".format(i)
    chunk.export(chunkfile, format="wav")
    y, sr = librosa.load(chunkfile)
    mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13))
    #print(mfcc.shape)
    #mfcc_array.append(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=39))
    if mfcc.shape[0]==13 and mfcc.shape[1]==44:
        a = []
        a.append(mfcc[0])
        a.append(mfcc[12])
        mfcc_array.append(a)
    #mfcc_array.append(np.pad(mfcc, (0,-mfcc.shape[0]), 'constant', constant_values=(0)))
    
mfcc_array = np.array(mfcc_array)

print(mfcc_array.shape)

#mfcc_array = tf.keras.utils.normalize(mfcc_array, order=2)
#mfcc_array = StandardScaler().fit_transform(mfcc_array)

z = mfcc_array.reshape(mfcc_array.shape[0], 2, 44, 1)

s=0
i=0
t=0

for f in z:
    #pc = model.predict_classes(f.reshape(1,39,44,1), batch_size=1)
    pp = model.predict(f.reshape(1,2,44,1), batch_size=1)
    pp = np.around(pp,decimals=2)
    #print("chunk",str(i),"-->",pc,"-->",pp)
    if pp[0][0] == 1.0:
        s+=1
    t+=pp[0][0]
    i+=1
    print("chunk",str(i),"-->",pp)

print(s/len(z)*100)
print(s)
print(len(z))
print(t/len(z)*100)


In [169]:
from keras.models import load_model
model1 = load_model('/home/mansi/anaconda3/beproject/stutter_det/models/rcnn-3d-select2-bkgn_rep_1_90v.h5')
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 20, 32)         192       
_________________________________________________________________
activation_4 (Activation)    (None, 3, 20, 32)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 8, 32)          5152      
_________________________________________________________________
activation_5 (Activation)    (None, 3, 8, 32)          0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 24, 32)            0         
_________________________________________________________________
gru1 (GRU)                   (None, 24, 32)            6240      
_________________________________________________________________
gru2 (GRU)                   (None, 32)                6240      
__________

In [171]:
import librosa
#import keras
import numpy as np
import tensorflow as tf
from pydub import AudioSegment
#from pydub.silence import split_on_silence
#import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
np.set_printoptions(threshold=np.nan)

arr = ['M_0760_15y3m_1',
       'M_1202_10y7m_1',
       'M_1215_11y0m_1',
       'M_1202_10y11m_1',
       'M_1202_10y7m_1',
       'M_0760_12y3m_1',
       'M_0077_11y2m_1',
       'M_0061_18y5m_2',
       'M_0061_18y5m_1',
       'F_1211_11y3m_1',
       'F_0101_14y8m_1',
       'M_1202_10y11m_1',
       'M_1216_11y1m_1',
       'M_0048_11y1m_2',
       'M_0048_11y5m_1',
       'M_0048_11y6m_1',
       'M_0053_10y8m_1',
       'M_0064_11y11m_1',
       'M_0064_11y5m_1'
      ]
arr2 = ["M_1206_12y3m_1",
"M_1202_10y7m_1",
"M_1023_11y6m_1",
"M_1022_10y9m_2",
"M_1213_11y3m_1",
"M_0253_14y6m_1",
"M_0104_15y4m_1",
"M_1022_10y10m_1",
"M_0880_15y2m_1",
"M_0760_12y3m_1",
"M_1215_11y0m_1",
"M_1214_11y8m_1",
"M_1207_13y5m_1",
"M_0760_15y3m_1",
"M_0760_12y3m_1",
"M_1023_10y10m_1",
"M_0545_15y3m_1",
"M_0541_12y3m_1",
"M_0104_18y1m_1",
"M_1216_11y1m_1",
"M_1205_13y1m_1",
"M_1040_12y6m_1",
"M_0880_14y6m_2"]

files = os.listdir("recordings")

for audio in arr2 :
    #if audio.endswith('.wav'):
    print(audio)
    #sound_file = AudioSegment.from_wav("/home/mansi/anaconda3/beproject/stutter_det/recordings/"+audio)
    sound_file = AudioSegment.from_wav("/home/mansi/anaconda3/beproject/stutter_det/test_rep/"+audio+'.wav')
    audio_chunks = sound_file[::1000]

    mfcc_array = []

    for i, chunk in enumerate(audio_chunks):
        chunkfile = "chunks_test/chunk{0}.wav".format(i)
        chunk.export(chunkfile, format="wav")
        y, sr = librosa.load(chunkfile)
        z = np.array(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13))
        a = []
        a.append(np.pad(z[0][1:], (0,44-z[0][1:].shape[0]), 'constant', constant_values=(0)))
        a.append(np.pad(z[2][1:], (0,44-z[0][1:].shape[0]), 'constant', constant_values=(0)))
        a.append(np.pad(z[5][1:], (0,44-z[0][1:].shape[0]), 'constant', constant_values=(0)))
        mfcc_array.append(a)

    mfcc_array = np.array(mfcc_array)
    z = mfcc_array.reshape(mfcc_array.shape[0], 3, 44, 1)
    s=0
    i=0
    t=0
    for f in z:
        pp = model1.predict(f.reshape(1,3,44,1), batch_size=1)
        pp = np.around(pp,decimals=2)
        if pp[0][0] > 0.98:
            s+=1
        t+=pp[0][0]
        #print("chunk",str(i),"-->",pp)
        i+=1
    print(t/len(z)*100)
    print(s/len(z)*100)

M_1206_12y3m_1
74.8418605033
64.65116279069767
M_1202_10y7m_1
66.1626017085
55.28455284552846
M_1023_11y6m_1
31.9508197626
16.39344262295082
M_1022_10y9m_2
41.4609375934
25.0
M_1213_11y3m_1
50.4836958317
32.065217391304344
M_0253_14y6m_1
39.9230771313
24.519230769230766
M_0104_15y4m_1
21.9636364067
13.939393939393941
M_1022_10y10m_1
57.6370968739
48.38709677419355
M_0880_15y2m_1
45.5197369802
27.631578947368425
M_0760_12y3m_1
37.2881355869
25.423728813559322
M_1215_11y0m_1
45.7068063699
29.84293193717277
M_1214_11y8m_1
52.9937889334
36.64596273291926
M_1207_13y5m_1
65.5380435794
52.17391304347826
M_0760_15y3m_1
22.6962810232
11.363636363636363
M_0760_12y3m_1
37.2881355869
25.423728813559322
M_1023_10y10m_1
55.2651516331
40.909090909090914
M_0545_15y3m_1
60.4090909643
44.15584415584416
M_0541_12y3m_1
15.7462686731
9.701492537313433
M_0104_18y1m_1
16.7763157895
9.210526315789473
M_1216_11y1m_1
64.177966301
55.932203389830505
M_1205_13y1m_1
50.3467743135
37.903225806451616
M_1040_12y6m_1
